In [1]:

import random

import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns


from tqdm import tqdm
from faker import Faker
from itertools import combinations


ModuleNotFoundError: No module named 'faker'

In [2]:

!pip install faker



                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB ? eta -:--:--
     --                                       0.1/1.7 MB 581.0 kB/s eta 0:00:03
     --                                       0.1/1.7 MB 581.0 kB/s eta 0:00:03
     --                                       0.1/1.7 MB 581.0 kB/s eta 0:00:03
     --                                       0.1/1.7 MB 581.0 kB/s eta 0:00:03
     --                                       0.1/1.7 MB 581.0 kB/s eta 0:00:03
     ---                                      0.2/1.7 MB 392.8 kB/s eta 0:00:04
     -----                                    0.2/1.7 MB 550.0 kB/s eta 0:00:03
     -----                                    0.2/1.7 MB 480.3 kB/s eta 0:00:04
     ------                                   0.3/1.7 MB 570.1 kB/s eta 0:00:03
     ------                                   0.3/1.7 MB 570.1 kB/s